In [66]:
import math
import random
import numpy as np

In [67]:
def euc_2d(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

In [68]:
def cost(sol,cities):
    distance = 0 
    for i,c1 in enumerate(sol):
        if (i == len(sol)-1):
            c2 = sol[0]
        else:
            c2 = sol[i+1]
        distance += euc_2d(cities[c1],cities[c2])
    return distance

In [69]:
def random_permutation(cities):
    perm = np.random.permutation(len(cities))
    return perm

In [70]:
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
   [880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
   [1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
   [415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
   [835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
   [410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
   [685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
   [95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
   [830,610],[605,625],[595,360],[1340,725],[1740,245]]

In [71]:
random_permutation(berlin52)

array([39, 25, 27,  9, 41, 14, 23,  8, 35, 38, 43, 13, 16, 36,  3, 15, 33,
       24, 17, 26, 45, 11, 10,  0, 37, 22, 34, 42, 20, 21, 48, 50, 47, 29,
       40, 51, 31, 19, 49,  7,  2, 28,  6, 44, 32, 30,  1, 12,  5,  4, 18,
       46])

In [72]:
def initialise_pheromone_matrix(num_cities, init_pher):
    return [ [init_pher]* num_cities for p in range(num_cities)]

In [78]:
def calculate_choices(cities,last_city,exclude,pheromone,c_heur,c_hist):
    choices = []
#     print(last_city)
    for i,coord in enumerate(cities):
        if i in exclude:
            continue
        prob = {'city':i}
        
        prob['history'] = pheromone[last_city][i] ** c_hist
        prob['distance'] = euc_2d(cities[last_city],coord)
        prob['heuristic'] = (1.0/prob['distance'])** c_heur
        prob['prob'] = prob['history'] * prob['heuristic']
        choices.append(prob)
    return choices
    

In [74]:
def prob_select(choices):
    sum_ = 0
    sum_ += sum([ prob['prob'] for prob in choices])
    if sum_ == 0:
        return random.choice(choices)['city']
    v = random.random()
    for i,choice in enumerate(choices):
        v -= (choice['prob']/sum_)
        if v <= 0.0:
            return choice['city']
    return choices[-1]['city']

In [75]:
def greedy_select(choices):
    
    return sorted(choices, key=lambda e: (-e['prob']))[0]['city']

In [76]:
def stepwise_const(cities,phero,c_huer,c_greed):
    perm = []
    perm.append(random.randint(0,51))## 
    while len(perm) < 52:
        choices = calculate_choices(cities,perm[-1],perm,phero,c_heur,1.0)
        greedy = random.random() <= c_greed
        if greedy:
            next_city = greedy_select(choices)
        else:
            next_city = prob_select(choices)
        perm.append(next_city)
    return perm

In [30]:
def global_update_pheromone(phero,cand,decay):
    for i,x in enumerate(cand['vector']):
        if  i == len(cand['vector'])-1:
            y = cand['vector'][0]
        else:
            y = cand['vector'][i+1]
        value = ((1.0-decay)*phero[x][y]) + (decay*(1.0/cand['cost']))
        phero[x][y] = value
        phero[y][x] = value
    

In [46]:
def local_update_pheromone(pheromone, cand, c_local_phero, init_phero):
    for i,x in enumerate(cand['vector']):
        if  i == len(cand['vector'])-1:
            y = cand['vector'][0]
        else:
            y = cand['vector'][i+1]
        value = ((1.0-c_local_phero)*pheromone[x][y])+(c_local_phero*init_phero)
        pheromone[x][y] = value
        pheromone[y][x] = value
      

In [49]:
def search(cities, max_it, num_ants, decay, c_heur, c_local_phero, c_greed):
    best = {'vector':random_permutation(cities)}
    best['cost'] = cost(best['vector'],cities)
    init_pheromone = 1.0 / (len(cities)* best['cost'])
    pheromone = initialise_pheromone_matrix(len(cities), init_pheromone)
    n = 0 
    while n < max_it:
        solutions = []
        for i in range(num_ants):
            cand = {}
            cand['vector'] = stepwise_const(cities,pheromone,c_heur,c_greed)
            cand['cost'] = cost(cand['vector'],cities)
            if cand['cost'] < best['cost']:
                best = cand
            local_update_pheromone(pheromone,cand,c_local_phero,init_pheromone)
        global_update_pheromone(pheromone,best,decay)
        print(best)
        n+=1
    return best
                   

In [35]:
# problem configuration
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]
# algorithm configuration
max_it = 100
num_ants = 10
decay = 0.1
c_heur = 2.5
c_local_phero = 0.1
c_greed = 0.9



In [79]:
# execute the algorithm
best = search(berlin52, max_it, num_ants, decay, c_heur, c_local_phero, c_greed)

{'vector': [49, 19, 22, 31, 48, 37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4, 14, 5, 3, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 2, 17, 30, 21, 0, 15, 28, 29, 20, 16, 41, 6, 1], 'cost': 8429.84812898167}
{'vector': [24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 29, 20, 16, 41, 6, 1, 49, 19, 22, 31, 48, 0, 21, 30, 17, 2, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11], 'cost': 8123.003156838685}
{'vector': [31, 48, 0, 21, 30, 17, 2, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 29, 20, 16, 41, 6, 1, 49, 19, 22], 'cost': 8123.003156838684}
{'vector': [30, 17, 2, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 19, 49, 22, 31, 48, 0, 21, 20, 16, 41, 6, 1, 29], 'cost': 8110.864843873408}
{'vector': [22, 19, 49, 15, 45, 43, 33, 34, 35, 3

{'vector': [16, 2, 17, 30, 21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 41, 1, 6, 20], 'cost': 7572.853381670416}
{'vector': [16, 2, 17, 30, 21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 41, 1, 6, 20], 'cost': 7572.853381670416}
{'vector': [16, 2, 17, 30, 21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 41, 1, 6, 20], 'cost': 7572.853381670416}
{'vector': [48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 41, 1, 6, 20, 16, 2, 17, 30, 21, 0], 'cost': 7572.853381670415}
{'vector': [48, 31, 44, 18, 40, 7, 8, 9, 42, 32,

{'vector': [21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 1, 6, 41, 20, 16, 2, 17, 30], 'cost': 7549.891295339107}
{'vector': [21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 1, 6, 41, 20, 16, 2, 17, 30], 'cost': 7549.891295339107}
{'vector': [21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 1, 6, 41, 20, 16, 2, 17, 30], 'cost': 7549.891295339107}
{'vector': [21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 1, 6, 41, 20, 16, 2, 17, 30], 'cost': 7549.891295339107}
{'vector': [21, 0, 48, 31, 44, 18, 40, 7, 8, 9, 